In [1]:
import pandas as pd
import numpy as np
import datetime
import datetime as dt
import pytz
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import statsmodels.api as sm  
import matplotlib.pyplot as plt  
import seaborn as sns
from statsmodels.tsa.arima_model import ARIMA  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
import os
working_directory = os.getcwd()
print(working_directory)

/Users/louisaliparisa/Desktop


In [3]:
def dateparse(time_in_secs):
    return pytz.utc.localize(datetime.datetime.fromtimestamp(float(time_in_secs)))

path = working_directory + '/data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv'
bitcoin1 = pd.read_csv(path, parse_dates=[0], date_parser=dateparse)

In [4]:
bitcoin1.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 02:52:00+00:00,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,2011-12-31 02:53:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-12-31 02:54:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-31 02:55:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-12-31 02:56:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
bitcoin = bitcoin1.dropna().reset_index(drop=True)
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 02:52:00+00:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 10:50:00+00:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,2011-12-31 11:59:00+00:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,2011-12-31 12:00:00+00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,2011-12-31 23:16:00+00:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
3613764,2021-03-30 19:56:00+00:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
3613765,2021-03-30 19:57:00+00:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
3613766,2021-03-30 19:58:00+00:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
3613767,2021-03-30 19:59:00+00:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [7]:
bitcoin['Timestamp'] = bitcoin['Timestamp'].dt.tz_localize(None)
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 02:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 10:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,2011-12-31 11:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,2011-12-31 12:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,2011-12-31 23:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
3613764,2021-03-30 19:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
3613765,2021-03-30 19:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
3613766,2021-03-30 19:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
3613767,2021-03-30 19:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [8]:
bitcoin = bitcoin.groupby([pd.Grouper(key='Timestamp', freq='D')]).first().reset_index()
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2012-01-01,4.84,4.84,4.84,4.84,10.000000,48.400000,4.840000
2,2012-01-02,5.00,5.00,5.00,5.00,19.048000,95.240000,5.000000
3,2012-01-03,5.32,5.32,5.32,5.32,2.419173,12.870000,5.320000
4,2012-01-04,4.93,4.93,4.93,4.93,9.680000,47.722400,4.930000
...,...,...,...,...,...,...,...,...
3373,2021-03-26,52449.23,52537.81,52449.23,52527.35,2.742287,143985.370640,52505.587896
3374,2021-03-27,54821.66,54856.01,54821.66,54856.01,0.261643,14346.416748,54832.004555
3375,2021-03-28,56095.81,56158.59,56095.81,56149.01,0.898928,50459.981106,56133.517533
3376,2021-03-29,55361.25,55417.15,55361.25,55401.67,1.314104,72804.822995,55402.653850


In [9]:
bitcoin = bitcoin.set_index('Timestamp')

In [10]:
bitcoinx = bitcoin[['Close']]
bitcoinx

,Close
Timestamp,
2011-12-31,4.39
2012-01-01,4.84
2012-01-02,5.00
2012-01-03,5.32
2012-01-04,4.93
...,...
2021-03-26,52527.35
2021-03-27,54856.01
2021-03-28,56149.01


In [11]:
bitcoinx['Close'] = bitcoinx['Close'].fillna(method='ffill')

<ipython-input-11-8db6d492bfdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoinx['Close'] = bitcoinx['Close'].fillna(method='ffill')


In [13]:
bitcoinx.index = pd.to_datetime(bitcoinx.index)

In [14]:
#print(bitcoin.head(20))


splitdate = '2018-12-17'
predate = '2020-04-01'
bitcoin_train = bitcoinx.loc[bitcoinx.index > splitdate]
bitcoin_train = bitcoin_train.loc[bitcoin_train.index <= predate]
bitcoin_test = bitcoinx.loc[bitcoinx.index > predate]
#bitcoin_test = bitcoin_test.loc[bitcoin_test.index < '2020-04-25']

In [15]:
bitcoin_train = bitcoin_train.reset_index()
bitcoin_trainx = bitcoin_train[['Timestamp']]
bitcoin_trainx['Timestamp'] = pd.to_datetime(bitcoin_trainx['Timestamp'])
bitcoin_trainx['Timestamp']=bitcoin_trainx['Timestamp'].map(dt.datetime.toordinal)
bitcoin_trainx

<ipython-input-15-44349ceef573>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_trainx['Timestamp'] = pd.to_datetime(bitcoin_trainx['Timestamp'])
<ipython-input-15-44349ceef573>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_trainx['Timestamp']=bitcoin_trainx['Timestamp'].map(dt.datetime.toordinal)


,Timestamp
0,737046
1,737047
2,737048
3,737049
4,737050
...,...
466,737512
467,737513
468,737514
469,737515


In [16]:
bitcoin_trainy = bitcoin_train[['Close']]
bitcoin_trainy

,Close
0,3517.74
1,3715.25
2,3736.95
3,4017.46
4,3807.18
...,...
466,6109.16
467,6209.34
468,6017.46
469,6435.00


In [17]:
bitcoin_test= bitcoin_test.reset_index()

bitcoin_test

,Timestamp,Close
0,2020-04-02,6626.56
1,2020-04-03,6783.30
2,2020-04-04,6697.55
3,2020-04-05,6785.86
4,2020-04-06,6864.44
...,...,...
358,2021-03-26,52527.35
359,2021-03-27,54856.01
360,2021-03-28,56149.01
361,2021-03-29,55401.67


In [18]:
bitcoin_testx = bitcoin_test[['Timestamp']]
bitcoin_testx['Timestamp'] = pd.to_datetime(bitcoin_testx['Timestamp'])
bitcoin_testx['Timestamp']=bitcoin_testx['Timestamp'].map(dt.datetime.toordinal)
bitcoin_testx

<ipython-input-18-f3d0816fa189>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_testx['Timestamp'] = pd.to_datetime(bitcoin_testx['Timestamp'])
<ipython-input-18-f3d0816fa189>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_testx['Timestamp']=bitcoin_testx['Timestamp'].map(dt.datetime.toordinal)


,Timestamp
0,737517
1,737518
2,737519
3,737520
4,737521
...,...
358,737875
359,737876
360,737877
361,737878


In [20]:
bitcoin_testy = bitcoin_test[['Close']]
bitcoin_testy

,Close
0,6626.56
1,6783.30
2,6697.55
3,6785.86
4,6864.44
...,...
358,52527.35
359,54856.01
360,56149.01
361,55401.67


In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
reg = LinearRegression()

In [23]:
reg = reg.fit(bitcoin_trainx,bitcoin_trainy)

In [24]:
bitcoin_predy = reg.predict(bitcoin_testx)

In [25]:
mean_squared_log_error(bitcoin_test['Close'], bitcoin_predy)

0.4016838940072041